# Transferencia de conocimiento sobre redes entrenadas con plantvillage

Realizamos fine tune y feature extraction sobre las redes entrenadas.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import scipy 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import IPython.display as display
import time

keras = tf.keras
print(tf.__version__)

2.0.0


In [3]:
data_dir = './dataset/trigo/enfermo/'
data_dir = pathlib.Path(data_dir)

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png')))
image_count

420

In [16]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  validation_split=0.15,                            
                                                                  zoom_range=0.2,
                                                                  horizontal_flip=True,
                                                                  vertical_flip=True,
                                                                  rotation_range=20
                                                                 )

In [17]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil((image_count*0.85)/BATCH_SIZE)
int(STEPS_PER_EPOCH)

12

In [18]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != ".ipynb_checkpoints"])
CLASS_NAMES

array(['mancha_foliar', 'oidio', 'roya_amarilla', 'roya_anaranjada'],
      dtype='<U15')

In [19]:
train_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "training",
                                                     classes = list(CLASS_NAMES))

Found 358 images belonging to 4 classes.


In [20]:
valid_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "validation",
                                                     classes = list(CLASS_NAMES))

Found 62 images belonging to 4 classes.


## Xception

In [21]:
base_model_x = tf.keras.models.load_model('models/plant_village_Xception_trained_01.h5')

In [24]:
global_average_layer_x = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_x = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")

In [25]:
base_model_x.layers[0].trainable = False

In [26]:
model_x = tf.keras.Sequential([
  base_model_x.layers[0],
  global_average_layer_x,
  prediction_layer_x
])

In [28]:
base_learning_rate = 0.003
model_x.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
model_x.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 8,196
Non-trainable params: 20,861,480
_________________________________________________________________


### Feature Extraction

In [30]:
total_epochs = 50
start_time = time.time()
history_x = model_x.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/50
12/12 [==============================] - 39s 3s/step - loss: 2.0208 - accuracy: 0.2821 - val_loss: 2.0694 - val_accuracy: 0.2742
Epoch 2/50
12/12 [==============================] - 33s 3s/step - loss: 1.3360 - accuracy: 0.4721 - val_loss: 2.1437 - val_accuracy: 0.2097
Epoch 3/50
12/12 [==============================] - 33s 3s/step - loss: 1.0859 - accuracy: 0.5754 - val_loss: 2.0681 - val_accuracy: 0.1774
Epoch 4/50
12/12 [==============================] - 33s 3s/step - loss: 1.0434 - accuracy: 0.6006 - val_loss: 1.5869 - val_accuracy: 0.2419
Epoch 5/50
12/12 [==============================] - 33s 3s/step - loss: 1.0687 - accuracy: 0.5838 - val_loss: 2.2122 - val_accuracy: 0.2581
Epoch 6/50
12/12 [==============================] - 34s 3s/step - loss: 1.0273 - accuracy: 0.6061 - val_loss: 2.6992 - val_accuracy: 0.1935
Epoch 7/50
12/12 [==============================] - 33s 3s/step - loss: 1.1494 - accuracy: 0.5978 - val_loss: 1.9108 - val_accuracy: 0.2581
Epoch 8/50
12/12 [==

### Fine Tune

In [31]:
base_model_x = tf.keras.models.load_model('models/plant_village_Xception_trained_01.h5')

In [32]:
for layer in base_model_x.layers[0].layers[:105]:
    layer.traianble = False
for layer in base_model_x.layers[0].layers[106:]:
    layer.traianble = True

In [33]:
global_average_layer_x = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_x = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")

In [34]:
model_x = tf.keras.Sequential([
  base_model_x.layers[0],
  global_average_layer_x,
  prediction_layer_x
])

In [35]:
base_learning_rate = 0.003
model_x.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model_x.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 20,815,148
Non-trainable params: 54,528
_________________________________________________________________


In [37]:
total_epochs = 50
start_time = time.time()
history_x_fine = model_x.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/50
12/12 [==============================] - 40s 3s/step - loss: 2.8083 - accuracy: 0.3101 - val_loss: 12.7266 - val_accuracy: 0.2097
Epoch 2/50
12/12 [==============================] - 34s 3s/step - loss: 1.7392 - accuracy: 0.3156 - val_loss: 11.4170 - val_accuracy: 0.2903
Epoch 3/50
12/12 [==============================] - 33s 3s/step - loss: 1.6447 - accuracy: 0.3492 - val_loss: 12.2565 - val_accuracy: 0.2419
Epoch 4/50
12/12 [==============================] - 34s 3s/step - loss: 2.0104 - accuracy: 0.3240 - val_loss: 11.9878 - val_accuracy: 0.2581
Epoch 5/50
12/12 [==============================] - 33s 3s/step - loss: 1.7687 - accuracy: 0.3184 - val_loss: 11.9710 - val_accuracy: 0.2581
Epoch 6/50
12/12 [==============================] - 34s 3s/step - loss: 1.6263 - accuracy: 0.3743 - val_loss: 12.2397 - val_accuracy: 0.2419
Epoch 7/50
12/12 [==============================] - 33s 3s/step - loss: 1.5302 - accuracy: 0.4078 - val_loss: 11.4338 - val_accuracy: 0.2903
Epoch 8/50
12

## MobileNet

In [38]:
base_model_m = tf.keras.models.load_model('models/plant_village_MobileNetV2_trained_01.h5')

In [39]:
base_model_m.layers[0].trainable = False

In [40]:
global_average_layer_m = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_m = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")

In [41]:
model_m = tf.keras.Sequential([
  base_model_m.layers[0],
  global_average_layer_m,
  prediction_layer_m
])

In [42]:
base_learning_rate = 0.003
model_m.compile(optimize r=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model_m.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


### Feature Extraction

In [44]:
total_epochs = 50
start_time = time.time()
history_m = model_m.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/50
12/12 [==============================] - 40s 3s/step - loss: 1.5208 - accuracy: 0.3128 - val_loss: 2.1296 - val_accuracy: 0.2258
Epoch 2/50
12/12 [==============================] - 34s 3s/step - loss: 1.2534 - accuracy: 0.4721 - val_loss: 2.1740 - val_accuracy: 0.2903
Epoch 3/50
12/12 [==============================] - 34s 3s/step - loss: 1.2003 - accuracy: 0.4581 - val_loss: 1.7956 - val_accuracy: 0.3387
Epoch 4/50
12/12 [==============================] - 34s 3s/step - loss: 1.1486 - accuracy: 0.5196 - val_loss: 2.2608 - val_accuracy: 0.3065
Epoch 5/50
12/12 [==============================] - 34s 3s/step - loss: 1.0972 - accuracy: 0.5503 - val_loss: 2.2841 - val_accuracy: 0.3065
Epoch 6/50
12/12 [==============================] - 34s 3s/step - loss: 1.1647 - accuracy: 0.5196 - val_loss: 2.6835 - val_accuracy: 0.3710
Epoch 7/50
12/12 [==============================] - 34s 3s/step - loss: 1.2617 - accuracy: 0.4693 - val_loss: 2.4573 - val_accuracy: 0.2258
Epoch 8/50
12/12 [==

### Fine tune

In [ ]:
base_model_m = tf.keras.models.load_model('models/plant_village_MobileNetV2_trained_01.h5')

In [ ]:
for i, layer in enumerate(base_model_m.layers[0].layers):
    print(i, layer.name)

In [ ]:
for layer in base_model_m.layers[0].layers[:125]:
    layer.traianble = False
for layer in base_model_m.layers[0].layers[126:]:
    layer.traianble = True

In [ ]:
global_average_layer_m = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_m = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_m = tf.keras.Sequential([
  base_model_m.layers[0],
  global_average_layer_m,
  prediction_layer_m
])

base_learning_rate = 0.003
model_m.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_m.summary()

In [ ]:
total_epochs = 6 0
start_time = time.time()
history_m = model_m.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

## ResNet

In [51]:
base_model_r = tf.keras.models.load_model('models/plant_village_ResNet50_trained_01.h5')
base_model_r.layers[0].trainable = False

In [52]:
global_average_layer_r = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_r = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_r = tf.keras.Sequential([
  base_model_r.layers[0],
  global_average_layer_r,
  prediction_layer_r
])


In [54]:
base_learning_rate = 0.003
model_r.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [55]:
model_r.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 8196      
Total params: 23,572,996
Trainable params: 8,196
Non-trainable params: 23,564,800
_________________________________________________________________


### Feature Extraction

In [56]:
total_epochs = 50
start_time = time.time()
history_r = model_r.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/50
12/12 [==============================] - 44s 4s/step - loss: 1.6979 - accuracy: 0.3017 - val_loss: 2.0315 - val_accuracy: 0.3387
Epoch 2/50
12/12 [==============================] - 36s 3s/step - loss: 1.2376 - accuracy: 0.4469 - val_loss: 1.8354 - val_accuracy: 0.3548
Epoch 3/50
12/12 [==============================] - 36s 3s/step - loss: 1.1601 - accuracy: 0.5363 - val_loss: 1.8970 - val_accuracy: 0.4194
Epoch 4/50
12/12 [==============================] - 35s 3s/step - loss: 1.1536 - accuracy: 0.5307 - val_loss: 1.7359 - val_accuracy: 0.3871
Epoch 5/50
12/12 [==============================] - 36s 3s/step - loss: 1.0350 - accuracy: 0.5978 - val_loss: 1.6387 - val_accuracy: 0.4194
Epoch 6/50
12/12 [==============================] - 36s 3s/step - loss: 1.0959 - accuracy: 0.5419 - val_loss: 1.8519 - val_accuracy: 0.4516
Epoch 7/50
12/12 [==============================] - 36s 3s/step - loss: 1.1152 - accuracy: 0.5670 - val_loss: 1.5666 - val_accuracy: 0.5161
Epoch 8/50
12/12 [==

### Fine tune 

In [57]:
base_model_r = tf.keras.models.load_model('models/plant_village_ResNet50_trained_01.h5')
for layer in base_model_r.layers[0].layers[:151]:
    layer.traianble = False
for layer in base_model_r.layers[0].layers[152:]:
    layer.traianble = True

In [58]:
global_average_layer_r = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_r = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_r = tf.keras.Sequential([
  base_model_r.layers[0],
  global_average_layer_r,
  prediction_layer_r
])

In [59]:
base_learning_rate = 0.003
model_r.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model_r.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 8196      
Total params: 23,572,996
Trainable params: 23,527,556
Non-trainable params: 45,440
_________________________________________________________________


In [61]:
total_epochs = 50
start_time = time.time()
history_r = model_r.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/50
12/12 [==============================] - 49s 4s/step - loss: 2.3721 - accuracy: 0.2821 - val_loss: 11.9543 - val_accuracy: 0.2581
Epoch 2/50
12/12 [==============================] - 37s 3s/step - loss: 1.6598 - accuracy: 0.3603 - val_loss: 11.9710 - val_accuracy: 0.2581
Epoch 3/50
12/12 [==============================] - 39s 3s/step - loss: 1.4321 - accuracy: 0.3743 - val_loss: 11.9375 - val_accuracy: 0.2581
Epoch 4/50
12/12 [==============================] - 39s 3s/step - loss: 1.3195 - accuracy: 0.4078 - val_loss: 12.7266 - val_accuracy: 0.2097
Epoch 5/50
12/12 [==============================] - 41s 3s/step - loss: 1.2645 - accuracy: 0.4218 - val_loss: 12.7434 - val_accuracy: 0.2097
Epoch 6/50
12/12 [==============================] - 34s 3s/step - loss: 1.3293 - accuracy: 0.4106 - val_loss: 12.3326 - val_accuracy: 0.1935
Epoch 7/50
12/12 [==============================] - 35s 3s/step - loss: 1.4060 - accuracy: 0.4581 - val_loss: 12.0046 - val_accuracy: 0.2581
Epoch 8/50
12

## VGG19

In [62]:
base_model_v = tf.keras.models.load_model('models/plant_village_VGG19_trained_01.h5')
base_model_v.traianble = False

In [63]:
global_average_layer_v = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_v = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_v = tf.keras.Sequential([
  base_model_v.layers[0],
  global_average_layer_v,
  prediction_layer_v
])

In [64]:
base_learning_rate = 0.003
model_v.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [65]:
model_r.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 8196      
Total params: 23,572,996
Trainable params: 23,527,556
Non-trainable params: 45,440
_________________________________________________________________


### Feature Extraction

In [66]:
total_epochs = 60
start_time = time.time()
history_r = model_r.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/60
12/12 [==============================] - 41s 3s/step - loss: 0.9644 - accuracy: 0.5894 - val_loss: 10.4447 - val_accuracy: 0.2258
Epoch 2/60
12/12 [==============================] - 36s 3s/step - loss: 0.8539 - accuracy: 0.6508 - val_loss: 8.7259 - val_accuracy: 0.1935
Epoch 3/60
12/12 [==============================] - 35s 3s/step - loss: 0.7430 - accuracy: 0.6927 - val_loss: 6.7838 - val_accuracy: 0.3710
Epoch 4/60
12/12 [==============================] - 36s 3s/step - loss: 0.7372 - accuracy: 0.6927 - val_loss: 5.5633 - val_accuracy: 0.2903
Epoch 5/60
12/12 [==============================] - 36s 3s/step - loss: 0.9697 - accuracy: 0.6257 - val_loss: 2.1376 - val_accuracy: 0.2742
Epoch 6/60
12/12 [==============================] - 37s 3s/step - loss: 0.9807 - accuracy: 0.6927 - val_loss: 1.9592 - val_accuracy: 0.3387
Epoch 7/60
12/12 [==============================] - 36s 3s/step - loss: 0.8196 - accuracy: 0.6732 - val_loss: 3.8492 - val_accuracy: 0.2419
Epoch 8/60
12/12 [=

12/12 [==============================] - 36s 3s/step - loss: 0.5527 - accuracy: 0.7933 - val_loss: 1.2650 - val_accuracy: 0.5968
Epoch 60/60
12/12 [==============================] - 36s 3s/step - loss: 0.5192 - accuracy: 0.7989 - val_loss: 1.8053 - val_accuracy: 0.5484
took: 35.99931418498357


### Fine tune

In [67]:
base_model_v = tf.keras.models.load_model('models/plant_village_VGG19_trained_01.h5')
for layer in base_model_v.layers[0].layers[:153]:
    layer.traianble = False
for layer in base_model_v.layers[0].layers[154:]:
    layer.traianble = True

In [68]:
global_average_layer_v = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer_v = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
model_v = tf.keras.Sequential([
  base_model_v.layers[0],
  global_average_layer_v,
  prediction_layer_v
])

In [69]:
base_learning_rate = 0.003
model_v.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
model_r.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 8196      
Total params: 23,572,996
Trainable params: 23,527,556
Non-trainable params: 45,440
_________________________________________________________________


In [71]:
total_epochs = 60
start_time = time.time()
history_r = model_r.fit_generator(train_data_gen,
                         epochs=total_epochs,
                         steps_per_epoch=train_data_gen.__len__(),
                         validation_data=valid_data_gen,
                         validation_steps= valid_data_gen.__len__(),
                        )
duration = time.time() - start_time
print('took: ' + str(duration/60))

Epoch 1/60
12/12 [==============================] - 40s 3s/step - loss: 0.6025 - accuracy: 0.7793 - val_loss: 3.6595 - val_accuracy: 0.3548
Epoch 2/60
12/12 [==============================] - 36s 3s/step - loss: 0.6479 - accuracy: 0.7849 - val_loss: 2.4003 - val_accuracy: 0.3871
Epoch 3/60
12/12 [==============================] - 37s 3s/step - loss: 0.5889 - accuracy: 0.7821 - val_loss: 4.2935 - val_accuracy: 0.3387
Epoch 4/60
12/12 [==============================] - 37s 3s/step - loss: 0.9088 - accuracy: 0.7598 - val_loss: 5.3115 - val_accuracy: 0.3548
Epoch 5/60
12/12 [==============================] - 35s 3s/step - loss: 0.5893 - accuracy: 0.7626 - val_loss: 2.0213 - val_accuracy: 0.4355
Epoch 6/60
12/12 [==============================] - 35s 3s/step - loss: 0.5488 - accuracy: 0.7933 - val_loss: 4.6207 - val_accuracy: 0.3226
Epoch 7/60
12/12 [==============================] - 36s 3s/step - loss: 0.7510 - accuracy: 0.7374 - val_loss: 1.0977 - val_accuracy: 0.5161
Epoch 8/60
12/12 [==

12/12 [==============================] - 37s 3s/step - loss: 0.4413 - accuracy: 0.8575 - val_loss: 1.7120 - val_accuracy: 0.5645
Epoch 60/60
12/12 [==============================] - 36s 3s/step - loss: 0.4021 - accuracy: 0.8408 - val_loss: 1.9667 - val_accuracy: 0.4839
took: 35.934785441557565
